# Machine Learning Analysis and Classification of STV Data

In [7]:
#import python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as scp

## for machine learning
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition

#set mathplot to inline (for jupyter)
%matplotlib inline 

## Read in Data

In [5]:
#read vote data from csv
#input file
constituency="DublinWest2002"
in_csv='../data/'+constituency+'.csv'
#in_csv ='../data/Meath2002.csv'
#in_csv ='../data/DublinNorth2002.csv'
#in_csv ='../data/DublinWest2002.csv'

#read in data (setting 1st row as header)
df = pd.read_csv(in_csv, na_values=["Missing"], header=[0])

#set column names as first line
#drop the numbers column (#df=df.drop(['No.'], 1))
df = df.drop(df.columns[[0]], axis=1)  # df.columns is zero-based pd.Index
#reset index to start a 1 and not 0
df.index = df.index + 1

In [6]:
#print input data frame details
print("Print Data Frame (df)")
print(df.head())
print("Print Column names")
print(df.columns)

Print Data Frame (df)
   GP   LB   FF   SP  FF.1   SF   PD   NP   FG
1 NaN  4.0  NaN  3.0   NaN  NaN  1.0  5.0  2.0
2 NaN  NaN  2.0  NaN   1.0  4.0  3.0  NaN  NaN
3 NaN  NaN  3.0  NaN   1.0  NaN  2.0  NaN  NaN
4 NaN  2.0  NaN  NaN   NaN  NaN  3.0  NaN  1.0
5 NaN  2.0  1.0  NaN   NaN  NaN  NaN  NaN  NaN
Print Column names
Index(['GP', 'LB', 'FF', 'SP', 'FF.1', 'SF', 'PD', 'NP', 'FG'], dtype='object')


## Data Analysis

In [ ]:
#Exploratory Data Analysis

## Feature Engineering

In [ ]:
#Feature Engineering

In [ ]:
#Feature Extraction

In [ ]:
#Split Data Set into Test and Train

## Model Selection

In [ ]:
#Model Selection
# Logistic Regression, Random Forrest, CNN etc.